In [1]:
!pip install -U transformers datasets accelerate
!pip install -U torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 72.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 36.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 82.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 47.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Found existing installation: torch 2.7.0
Uninstalling torch-2.7.0:
  Successfully uninstalled torch-2.7.0
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 42.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 11.5 M

In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [8]:
import os
import pandas as pd
import numpy as np
from PIL import Image, ImageDraw

import torch


from torch.utils.data import Dataset
from torchvision import transforms




from datasets import Dataset as HFDataset
from transformers import (
    ViTForImageClassification,
    ViTImageProcessor,
    TrainingArguments,
    Trainer
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# config
IMG_DIR = "dataset"
CSV_FILE = "date.csv"
MODEL_NAME = "google/vit-base-patch16-224"
NUM_LABELS = 3

# procesare fisier csv
df = pd.read_csv(CSV_FILE, header=None,
                 names=["id", "density", "type", "label", "x", "y", "radius"])

df["filename"] = df["id"].apply(lambda x: f"{x}.pgm")
df = df[df["type"].notna()].reset_index(drop=True)
df["x"] = pd.to_numeric(df["x"], errors='coerce')
df["y"] = pd.to_numeric(df["y"], errors='coerce')
df["radius"] = pd.to_numeric(df["radius"], errors='coerce')

# labeluri: 0 = normal, 1 = benign, 2 = malign
df["class"] = df["label"].apply(lambda x: 1 if x == "B" else (2 if x == "M" else 0))

processor = ViTImageProcessor.from_pretrained(MODEL_NAME)

class MiasDataset(Dataset):
    def __init__(self, dataframe, img_dir, processor):
        self.df = dataframe
        self.img_dir = img_dir
        self.processor = processor

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, row["filename"])
        image = Image.open(img_path).convert("RGB").resize((224, 224))

        # normalizare
        x = row["x"] / image.width if pd.notna(row["x"]) else 0
        y = row["y"] / image.height if pd.notna(row["y"]) else 0

        inputs = processor(images=image, return_tensors="pt")
        inputs = {k: v.squeeze(0) for k, v in inputs.items()}
        inputs["label"] = torch.tensor(row["class"]).long()
        inputs["coord"] = torch.tensor([x, y], dtype=torch.float32)

        return inputs

# train/test
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df, test_size=0.1, stratify=df["class"], random_state=42)

train_dataset = MiasDataset(df_train, IMG_DIR, processor)
test_dataset = MiasDataset(df_test, IMG_DIR, processor)

# folosim ViT
model = ViTForImageClassification.from_pretrained(
    MODEL_NAME,
    num_labels=NUM_LABELS,
    id2label={0: "normal", 1: "benign", 2: "malign"},
    label2id={"normal": 0, "benign": 1, "malign": 2},
    ignore_mismatched_sizes=True
)

model = model.to(device)

# setam training args
training_args = TrainingArguments(
    output_dir="./vit-mias-output",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    eval_strategy="epoch",
    save_strategy="epoch",
    num_train_epochs=6,
    learning_rate=2e-5,
    logging_dir="./logs",
    logging_steps=10,
)

from transformers import DefaultDataCollator
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    labels = p.label_ids

    acc = accuracy_score(labels, preds)
    precision, recall, f1, _ = precision_recall_fscore_support(
        labels, preds, average='weighted', zero_division=0
    )

    return {
        "accuracy": acc,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=processor,
    data_collator=DefaultDataCollator(),
    compute_metrics=compute_metrics
)

#  antrenam modelul
trainer.train()

# metrici finale
metrics = trainer.evaluate()
print("final metrics:", metrics)

Using device: cpu


Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([3]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([3, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
<ipython-input-8-1884d4553959>:125: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: Use

Epoch,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
1,0.924600,0.920185,0.636364,0.404959,0.636364,0.494949
2,0.856800,0.940273,0.575758,0.416928,0.575758,0.483636
3,0.624400,0.944559,0.545455,0.409091,0.545455,0.467532
4,0.379200,0.909107,0.545455,0.409091,0.545455,0.467532
5,0.243700,0.946993,0.606061,0.511267,0.606061,0.552696
6,0.144700,0.952168,0.636364,0.532660,0.636364,0.574495


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument i

Final test metrics: {'eval_loss': 0.9521682262420654, 'eval_accuracy': 0.6363636363636364, 'eval_precision': 0.5326599326599327, 'eval_recall': 0.6363636363636364, 'eval_f1': 0.5744949494949495, 'eval_runtime': 23.2098, 'eval_samples_per_second': 1.422, 'eval_steps_per_second': 0.215, 'epoch': 6.0}


In [9]:
trainer.save_model("./vit-mias-final")

In [14]:
import torch
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor

# config
IMG_PATH = "dataset/mdb023.pgm"
MODEL_CHECKPOINT = "./vit-mias-output/checkpoint-228"

# pt gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained(MODEL_CHECKPOINT)
model = model.to(device)
model.eval()

# predictie pt o imagine
def predict_image(img_path):
    image = Image.open(img_path).convert("RGB").resize((224, 224))
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax(-1).item()
        predicted_label = model.config.id2label[predicted_class_id]

    return predicted_label

predicted = predict_image(IMG_PATH)
print(f"Imaginea {IMG_PATH} este clasificată ca: {predicted}")


Imaginea dataset/mdb023.pgm este clasificată ca: malign


In [15]:
import torch
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor

# config
IMG_PATH = "dataset/mdb003.pgm"
MODEL_CHECKPOINT = "./vit-mias-output/checkpoint-228"

# pt gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained(MODEL_CHECKPOINT)
model = model.to(device)
model.eval()

# predictie pt o imagine
def predict_image(img_path):
    image = Image.open(img_path).convert("RGB").resize((224, 224))
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax(-1).item()
        predicted_label = model.config.id2label[predicted_class_id]

    return predicted_label

predicted = predict_image(IMG_PATH)
print(f"Imaginea {IMG_PATH} este clasificată ca: {predicted}")


Imaginea dataset/mdb003.pgm este clasificată ca: normal


In [22]:
import torch
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor

# config
IMG_PATH = "dataset/mdb025.pgm"
MODEL_CHECKPOINT = "./vit-mias-output/checkpoint-228"

# pt gpu
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained(MODEL_CHECKPOINT)
model = model.to(device)
model.eval()

# predictie pt o imagine
def predict_image(img_path):
    image = Image.open(img_path).convert("RGB").resize((224, 224))
    inputs = processor(images=image, return_tensors="pt").to(device)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        predicted_class_id = logits.argmax(-1).item()
        predicted_label = model.config.id2label[predicted_class_id]

    return predicted_label

predicted = predict_image(IMG_PATH)
print(f"Imaginea {IMG_PATH} este clasificată ca: {predicted}")


Imaginea dataset/mdb025.pgm este clasificată ca: benign


In [27]:
import torch
from PIL import Image
from transformers import ViTForImageClassification, ViTImageProcessor

image_id = input("Introdu numele imaginii (ex: mdb023): ").strip()

IMG_PATH = f"dataset/{image_id}.pgm"
MODEL_CHECKPOINT = "./vit-mias-output/checkpoint-228"

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224")
model = ViTForImageClassification.from_pretrained(MODEL_CHECKPOINT)
model = model.to(device)
model.eval()

# functie de predictie
def predict_image(img_path):
    try:
        # verificare existență fișier
        if not os.path.exists(img_path):
            return f"eroare: imaginea '{img_path}' nu exista"

        image = Image.open(img_path).convert("RGB").resize((224, 224))
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            predicted_class_id = logits.argmax(-1).item()
            predicted_label = model.config.id2label[predicted_class_id]

        return f"Imaginea {img_path} este clasificată ca: {predicted_label}"

    except UnidentifiedImageError:
        return f"eroare: fisierul '{img_path}' nu este o imagine valida"
    except Exception as e:
        return f"eroare: {str(e)}"

predicted = predict_image(IMG_PATH)
print(f"{predicted}")


Introdu numele imaginii (ex: mdb023): mdb008
 Imaginea dataset/mdb008.pgm este clasificată ca: normal


In [18]:
model.save_pretrained("vit-mias-model")
processor.save_pretrained("vit-mias-model")


['vit-mias-model/preprocessor_config.json']

In [19]:
import shutil
shutil.make_archive("vit-mias-model", 'zip', "vit-mias-model")


'/content/vit-mias-model.zip'

In [20]:
from google.colab import files
files.download("vit-mias-model.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>